In [1]:
from io import BytesIO
import matplotlib.dates as dates
import requests
import pandas as pd
import re
import scipy.optimize
import numpy as np
from matplotlib import pyplot as plt
from openpyxl import load_workbook
import ipywidgets as widgets

%matplotlib ipympl

In [2]:
response = requests.get("https://www.arcgis.com/sharing/rest/content/items/e5fd11150d274bebaaf8fe2a7a2bda11/data")
io = BytesIO(response.content)

In [3]:
workbook = load_workbook(io, read_only=True)

In [4]:
header, *rows = workbook.worksheets[0].values
df = pd.DataFrame({c: b for c, b in zip(header, zip(*rows))})
df.tail()

,DateVal,CMODateCount,CumCases,DailyDeaths,CumDeaths
64,2020-04-04,3735,41903,708.0,4313.0
65,2020-04-05,5914,47817,621.0,4934.0
66,2020-04-06,3802,51608,439.0,5373.0
67,2020-04-07,3634,55242,786.0,6159.0
68,2020-04-08,5491,60733,938.0,7097.0


In [5]:
df = df.rename(columns={'CMODateCount': 'new cases', 'CumCases': 'cases', 'DateVal': 'date'})
df['cases'] = df['new cases'].cumsum()
df.tail()

,date,new cases,cases,DailyDeaths,CumDeaths
64,2020-04-04,3735,41908,708.0,4313.0
65,2020-04-05,5914,47822,621.0,4934.0
66,2020-04-06,3802,51624,439.0,5373.0
67,2020-04-07,3634,55258,786.0,6159.0
68,2020-04-08,5491,60749,938.0,7097.0


In [6]:
# df = df.append({'date': pd.to_datetime(pd.to_datetime('today').date()), 'new cases': 530-456, 'cases': 530}, ignore_index=True)
# df.tail()

In [20]:
df.plot('date', 'cases', kind='scatter', title='UK Cases');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
def fit_func(x, a, m, c):
    return a * np.exp(m*(x + c))

In [22]:
prediction_dates = df['date'][:-1]
prediction_days = prediction_dates.dt.dayofyear
popt, pcov = scipy.optimize.curve_fit(fit_func, prediction_days, df['cases'][:prediction_days.size], p0=(0.3, 0.13, -2))
popt

array([  0.560893  ,   0.13222997, -10.40240787])

In [23]:
f"Time to double is {np.log(2)/popt[1]} days"

'Time to double is 5.241982381785093 days'

In [24]:
def day_of_year_to_date(doy):
    return pd.to_datetime(doy-1, unit='D', origin=pd.Timestamp("2020-01-01"))

In [25]:
doy_next = prediction_days.to_numpy()[-1]+0
n_cases_after_prediction = fit_func(doy_next, *popt)
f"{n_cases_after_prediction:.0f} cases expected on {day_of_year_to_date(doy_next)}"

'60162 cases expected on 2020-04-07 00:00:00'

In [26]:
plt.plot(prediction_dates, fit_func(prediction_days, *popt), 'g--');

In [27]:
def predict(change):
    n_days = change['new']
    t_projection = prediction_days.to_numpy()[-1] + np.arange(n_days) + 1
    dt_projection = pd.to_datetime(t_projection-1, unit='D', origin=pd.Timestamp("2020-01-01"))
    line_predict.set_data(dt_projection, fit_func(t_projection, *popt))
    print(dt_projection, )
    
    ax = plt.gca()
    ax.relim()
    ax.autoscale()

In [28]:
line_predict, = plt.plot(df['date'], df['cases'], 'rx--');

In [29]:
n_days = widgets.IntSlider(min=0, max=40, description="Days")
n_days.observe(predict, 'value')
n_days

IntSlider(value=0, description='Days', max=40)

## Plot cases per day vs cases
For exponential growth a simple $\dot{N}$ vs $N$ plot should give
$$
\dot{N} = m N\,,
$$
where $m$ is the growth rate.
Given that the error on the number of cases is not constant, it follows that larger points are weighted more strongly when fitting with OLS. At the moment this simple model does not consider errors. We use `curve_fit` to fit non-linearly.

In [30]:
def proportional(x, m):
    return m*x

df_gt_100 = df.loc[df['cases'] > 100]

popt_linear_cases, pcov_linear_cases = scipy.optimize.curve_fit(proportional, 
                                                  df_gt_100['cases'], 
                                                  df_gt_100['new cases'],
                                                )

df_gt_100.plot("cases", "new cases", kind='scatter')
plt.plot(df_gt_100['cases'], proportional(df_gt_100['cases'], *popt_linear_cases));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Now let's plot on a log-log scale.

In [31]:
df_gt_100.plot("cases", "new cases", logx=True, logy=True, kind='scatter')
plt.loglog(df_gt_100['cases'], proportional(df_gt_100['cases'], *popt_linear_cases));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

This looks more linear, suggestion a relation
$$
\log{\dot{N}} = m\log{N} + c\,,
$$
which transforms to
$$
\dot{N} = N^m\cdot c\,,
$$
where $m=1$, $c=0$ for exponential growth. Let's fit the log-log plot with a linear function instead:

In [33]:
def linear(x, m, c): 
    return m*x + c

popt_loglog_cases, pcov_loglog_cases = scipy.optimize.curve_fit(
    linear, 
    np.log(df_gt_100['cases']), 
    np.log(df_gt_100['new cases'])
)

df.plot("cases", "new cases", kind='scatter', logx=True, logy=True)
plt.loglog(df['cases'], np.exp(linear(np.log(df['cases']), *popt_loglog_cases)));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Evidently there is some nonlinearity in the model, given that $m \neq 1$:

In [34]:
popt_loglog_cases

array([ 0.85595587, -0.63663948])